In [2]:
from selenium.webdriver import Chrome 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# options. Headless mode, selenium will be working, but chrome won't pop up. Will just scrape data in the background, without seeing it
# options = Options()
# options.add_argument("--headless")

driver = Chrome(ChromeDriverManager().install())
driver.get("https://coastradar.com/beaches-near-me/")


    



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/frankiejames/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60/chromedriver] found in cache
/var/folders/sf/lnrr_g5s6bd9cf8zjnzr2b6w0000gn/T/ipykernel_85396/2979753795.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(ChromeDriverManager().install())
